# The Impact of Litigations on GHG Emissions
* "litigation" from [Graham Institute](https://climate-laws.org/legislation_and_policies), takes the value of 1 when the first case related to environmental laws was solved in a given year, independently of the result
* "rule of law" from World Bank
* Sample excludes the US, and includes all countries for which there has been at least a case with a resolution: 23 countries, mostly HI and MI countries

##### Load R packages

In [1]:
options(conflicts.policy = list(warn = FALSE))
library("dplyr")
library("estimatr")
library("fixest")
library("ggplot2")
library("gmm")
library("lmtest")
library("plm")
library("readr")
library("rstatix")

Loading required package: sandwich

Loading required package: zoo



##### Read data

In [3]:
my_data <- read_csv("../data/litigclean_eps.csv", show_col_types = 'FALSE')

##### Define Country_ID as numerical

In [4]:
# egen id = group(country)
my_data <- my_data %>% group_by(country) %>% mutate(id = cur_group_id())

In [5]:
# xtset id year
plm::is.pbalanced(my_data$id, my_data$year)

[1] TRUE

##### Creation of log values

In [6]:
my_data$lnghg <- log(my_data$ghg)
my_data$lngdp <- log(my_data$gdp)
my_data$lngdp2 <- log(my_data$gdp)^2
my_data$lnpopulation <- log(my_data$population)
my_data$lnlaw <- log(my_data$ruleoflaw)

##### Definition of High-Income and Middle-Income countries

In [7]:
HI <- my_data$country %in% c('BEL', 'AUS', 'CAN', 'CHE', 'DEU', 'ESP', 'EST', 'FIN', 
                             'FRA', 'GBR', 'GRC', 'HRV', 'ITA', 'LTU', 'LUX', 'NLD', 
                             'NOR', 'NZL', 'POL', 'PRT', 'SVK', 'SWE', 'ROU')


In [8]:
MI <- my_data$country %in% c('BGR', 'CRI', 'FJI', 'IND', 'KEN', 
                             'PAK', 'NPL', 'PHL', 'UKR')

### Table 3. Estimation Results for the Whole Sample

(1) OLS

(2) TW-FE

In [9]:
# xtreg lnghg lngdp lngdp2 lnpop lnlaw litigation eps i.year, fe robust

modfe <- plm::plm(lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + litigation + eps,
                  index = c("id", "year"), model="within", effect='twoways', 
                  data = my_data)
summary(modfe, vcov=plm::vcovHC(modfe))

Twoways effects Within Model

Note: Coefficient variance-covariance matrix supplied: plm::vcovHC(modfe)

Call:
plm::plm(formula = lnghg ~ lngdp + lngdp2 + lnpopulation + lnlaw + 
    litigation + eps, data = my_data, effect = "twoways", model = "within", 
    index = c("id", "year"))

Unbalanced Panel: n = 18, T = 19-20, N = 359

Residuals:
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-0.1856773 -0.0348761 -0.0014917  0.0337854  0.1519308 

Coefficients:
              Estimate Std. Error t-value  Pr(>|t|)    
lngdp         2.345554   0.409284  5.7309 2.331e-08 ***
lngdp2       -0.094936   0.023563 -4.0291 7.020e-05 ***
lnpopulation  1.244627   0.243557  5.1102 5.581e-07 ***
lnlaw        -0.119600   0.018605 -6.4282 4.758e-10 ***
litigation   -0.031540   0.015878 -1.9863   0.04786 *  
eps          -0.027488   0.020243 -1.3579   0.17546    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    3.3802
Residual Sum of Squares: 1.0695
R-Squa